In [1]:
import pickle
import pandas as pd
import pyvis
import os

In [3]:
root_path = r"C:\Users\Joycelyn\Documents\Synapxe\07 GenAI for healthhub\Github backup\cluster experiments\weighted emb vs sim (658)"
emb_path_title = os.path.join(root_path,"nomic_weighted_emb (0.7 body, 0.3 title)")
emb_path_metadesc = os.path.join(root_path,"nomic_weighted_emb (0.7 body, 0.3 meta desc)")

with open(os.path.join(emb_path_title,"neo4j_predicted_clusters.pkl"), "rb") as f:
    title_results = pickle.load(f)

with open(os.path.join(emb_path_metadesc,"neo4j_predicted_clusters.pkl"), "rb") as f:
    desc_results = pickle.load(f)

In [16]:
title_results = pd.DataFrame(title_results)
title_results['count'] = title_results['titles'].apply(len)
title_results_filtered = title_results[title_results["count"] < 6]
print(title_results_filtered.shape)

(59, 4)


In [17]:
desc_results = pd.DataFrame(desc_results)
desc_results['count'] = desc_results['titles'].apply(len)
desc_results_filtered = desc_results[desc_results["count"] < 6]
print(desc_results_filtered.shape)

(57, 4)


In [19]:
save_path = os.path.join(root_path, "body_with_title_vs_metadesc_size5.xlsx")
with pd.ExcelWriter(save_path) as writer:
    title_results_filtered.to_excel(writer, sheet_name='title_results_filtered', index=False)
    desc_results_filtered.to_excel(writer, sheet_name='desc_results_filtered', index=False)